In [128]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import KFold

In [129]:
df = pd.read_csv('all_data_combined.csv')
df.head()

,subject_id,age,urea_n_min,urea_n_max,urea_n_mean,resprate_min,resprate_max,resprate_mean,glucose_min,glucose_max,...,oasis,lods,gender_F,gender_M,marital_status_DIVORCED,marital_status_MARRIED,marital_status_SEPARATED,marital_status_SINGLE,marital_status_WIDOWED,death
0,15057,58.831224,16.0,37.0,26.111111,10.0,52,20.104478,67.0,405.0,...,36.5,5.5,0,1,0,1,0,0,0,1
1,79262,81.618606,10.0,15.0,13.500000,11.0,31,18.704225,119.0,348.0,...,48.0,4.0,0,1,0,1,0,0,0,1
2,77191,79.102744,16.0,56.0,35.500000,20.0,31,25.744681,90.0,188.0,...,35.0,6.0,1,0,1,0,0,0,0,0
3,84966,88.232043,8.0,27.0,17.692308,10.0,51,17.770833,92.0,271.0,...,40.5,4.0,1,0,0,0,0,1,0,1
4,94997,90.000000,38.0,48.0,42.250000,4.0,40,26.360000,62.0,135.0,...,35.0,5.0,1,0,0,0,0,1,0,0


In [130]:
df.shape

(2152, 168)

In [131]:
def decisionTree(df,method,max_depth,impurity_decrease ):
    
    kf = KFold(n_splits=3, random_state=0)
    
    result = []
    
    for train, test in kf.split(df):

        train_data = df.iloc[train,:]
        test_data =  df.iloc[test,:]

        trainx = train_data.iloc[:,1:(train_data.shape[1]-2)]
        trainy = train_data.iloc[:,train_data.shape[1]-1]

        testx = test_data.iloc[:,1:(test_data.shape[1]-2)]
        testy = test_data.iloc[:,test_data.shape[1]-1]

        model = tree.DecisionTreeClassifier(criterion = method,max_depth=max_depth,
                                            min_impurity_decrease=impurity_decrease)
        model.fit(trainx, trainy)
        result.append(model.score(testx, testy))
        
    return np.average(result)

In [132]:
df_para = pd.read_csv('tree_params.csv')

In [133]:
scores =[]
for indx,row in df_para.iterrows():
    scores.append(decisionTree(df,row['method'],max_depth=row['max_depth'],impurity_decrease= row['min_impurity_decrease']))
df_para['scores']=scores

In [134]:
df_para.to_csv('tree_params.csv',index = False)
df_para

,method,max_depth,min_impurity_decrease,scores
0,entropy,10,0.01,0.743044
1,entropy,20,0.01,0.743044
2,entropy,50,0.01,0.743044
3,entropy,100,0.01,0.743044
4,gini,10,0.01,0.692844
5,gini,20,0.01,0.692844
6,gini,50,0.01,0.692844
7,gini,100,0.01,0.692844
